In [14]:
import pandas as pd
import env

from  sklearn.preprocessing import LabelEncoder

url = f'mysql+pymysql://{env.user}:{env.pw}@{env.host}/numbers'

In [3]:
df = pd.read_sql('SELECT * FROM numbers_with_more_groups', url)
df

,n,category,supergroup
0,1,a,one
1,2,b,two
2,3,c,one
3,4,a,two
4,5,b,one
5,6,c,two
6,7,a,one
7,8,b,two
8,9,c,one
9,10,a,two


In [4]:
supergroup_means = df.groupby('supergroup').n.transform('mean')
df['supergroup_mean'] = supergroup_means
df

,n,category,supergroup,supergroup_mean
0,1,a,one,5
1,2,b,two,6
2,3,c,one,5
3,4,a,two,6
4,5,b,one,5
5,6,c,two,6
6,7,a,one,5
7,8,b,two,6
8,9,c,one,5
9,10,a,two,6


### Exercise
Obtain the cars.csv file from the google classroom and read it into python with pandas.

Create a feature named gt_avg, which should be either 1 or 0. The value should indicate whether or not a given price is greater than the average price for that car's combination of year, make, and model.

Drop the Id, City, and Vin columns.

Encode the categorical features as necessary. You might wish to use a sklearn.preprocessing.LabelEncoder.

Split the data into training and test sets.

In [6]:
cars = pd.read_csv('./cars.csv')
cars.columns = [col.lower() for col in cars]
print('%d rows X %d columns' % cars.shape)
cars.head()

297899 rows X 9 columns


,id,price,year,mileage,city,state,vin,make,model
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


In [7]:
cars['avg_saleprice'] = cars.groupby(['make', 'model', 'year']).price.transform('mean')


In [8]:
cars['gt_avg'] = (cars.price > cars.avg_saleprice).astype(int)

In [9]:
cars.head()

,id,price,year,mileage,city,state,vin,make,model,avg_saleprice,gt_avg
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience,17291.768786,0
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD,16721.350598,0
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather,19080.632911,0
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD,16721.350598,0
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience,17291.768786,0


In [10]:
cars.drop(columns=['id', 'price', 'city', 'vin', 'avg_saleprice'], inplace=True)

In [11]:
cars.head()

,year,mileage,state,make,model,gt_avg
0,2015,18681,MO,Buick,EncoreConvenience,0
1,2015,27592,IN,Buick,EncoreFWD,0
2,2015,13650,NC,Buick,EncoreLeather,0
3,2015,25195,LA,Buick,EncoreFWD,0
4,2015,22800,NV,Buick,EncoreConvenience,0


### Predict whether a car's sales price will be higher than the average or not. (Linear regression problem becomes a classification problem)

In [16]:
for col in ['year', 'state', 'make', 'model']:
    le = LabelEncoder().fit(cars[col])
    cars[col] = le.transform(cars[col])
cars.head()

,year,mileage,state,make,model,gt_avg
0,18,18681,28,7,523,0
1,18,27592,19,7,525,0
2,18,13650,32,7,526,0
3,18,25195,22,7,525,0
4,18,22800,38,7,523,0


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X, y = cars.drop(columns = 'gt_avg'), cars.gt_avg

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree = DecisionTreeClassifier(max_depth=4)
cross_val_score(tree, X_train, y_train, cv=3).mean()

0.6414679489200495

In [20]:
tree = DecisionTreeClassifier(max_depth=2)
cross_val_score(tree, X_train, y_train, cv=3, scoring='precision').mean()

0.608612184084853